In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [ ]:
# This function was generated by ChatGPT
def divide_transcript_into_blocks(transcript, block_size):
    """
    Divide the given transcript into blocks of the given size and return an array containing the text of each block,
    the starting time, and end time of each block.
    """
    # Initialize variables to keep track of the current block's start and end time, and the current number of words and text
    block_start_time = 0
    block_end_time = 0
    current_word_count = 0
    current_text = ""
    blocks = []
    
    # Loop through each line of the transcript
    for line in transcript:
        # Calculate the length of the current line in words
        line_word_count = len(line['text'].split())

        # If adding the current line to the current block would exceed the maximum block size, add the current block to the list and start a new one
        if current_word_count + line_word_count > block_size:
            blocks.append({'text': current_text, 'start_time': block_start_time, 'end_time': block_end_time})
            block_start_time = line['start']
            block_end_time = line['start']
            current_word_count = 0
            current_text = ""

        # Add the current line to the current block and update the current block's end time
        current_word_count += line_word_count
        block_end_time = line['start'] + line['duration']
        current_text += line['text'] + " "

    # Add the final block to the list
    blocks.append({'text': current_text, 'start_time': block_start_time, 'end_time': block_end_time})
    
    return blocks

In [ ]:
import youtube_transcript_api

# Replace the VIDEO_ID with the ID of the YouTube video you want to retrieve the transcript for
video_id = 'VIDEO_ID'

# Retrieve the transcript for the YouTube video using the youtube_transcript_api library
transcript = youtube_transcript_api.YouTubeTranscriptApi.get_transcript(video_id)

# Define the block size (in number of words)
block_size = 3000

# Divide the transcript into blocks
blocks = divide_transcript_into_blocks(transcript, block_size)

# Print the blocks
for i, block in enumerate(blocks):
    print(f"Block {i+1} start time: {block['start_time']} s, end time: {block['end_time']} s")
    max_sentences = 5
    temperature=0

    prompt = f"""
        List {max_sentences} important aspects ot the text delimited by triple backticks.
        ```{block['text']}```
        """
    
    response = get_completion(prompt)
    print(response)